In [ ]:
import numpy as np
import pandas as pd
import re 
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup  
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

Загрузка обучающих и тестовых данных

In [ ]:
train = pd.read_csv("../input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip", header=0, delimiter="\t", quoting=3)
test = pd.read_csv("../input/word2vec-nlp-tutorial/testData.tsv.zip", header=0, delimiter="\t", quoting=3 )
train.head()

Очистка текста: html теги, знаки препинания.

In [ ]:
clean_train_reviews = []
for i in range(0, train["review"].size):
    # Удаление html тегов
    review_text = BeautifulSoup(train["review"][i]).get_text()      
    # Удаление непечатных символов (не букв)
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    # Перевод символов в нижний регистр, разделение на слова
    words = letters_only.lower().split()                             
    # Удалние stopwords
    stopwordss = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if not w in stopwordss]   
    # Объединение слов с добавлением пробела
    clean_train_reviews.append(" ".join(meaningful_words))

Получение вектора признаков из тренировочных данных (создание bag of words).

In [ ]:
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 

train_data_features = vectorizer.fit_transform(clean_train_reviews)

train_data_features = train_data_features.toarray()

Обучение алгоритма Random Forest

In [ ]:
clf = RandomForestClassifier(n_estimators=200) 
clf.fit(train_data_features, train["sentiment"])

Формирование результата: предсказание и формирование файла предсказаний

In [ ]:
clean_test_reviews = [] 

for i in range(0,len(test["review"])):
    review_text = BeautifulSoup(test["review"][i]).get_text()      
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    words = letters_only.lower().split()                             
    stopwordss = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if not w in stopwordss]  
    clean_test_reviews.append(" ".join(meaningful_words))

test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

result = clf.predict(test_data_features)

output = pd.DataFrame(data={"id":test["id"], "sentiment":result})

output.to_csv("submission.csv", index=False, quoting=3)